# 🧹 Processo de Limpeza e Qualidade dos Dados

## 🎯 Objetivo
Estabelecer um dataset confiável e consistente através de técnicas rigorosas de limpeza e validação de qualidade.

In [3]:
import pandas as pd

In [4]:
# Import the dataset into a pandas DataFrame
df = pd.read_csv('../dados/github_repos_completos.csv')

df.head()

,name,owner,stars,forks,language,created_at,updated_at,size_kb,watchers_count,open_issues,owner_type,owner_public_repos,owner_location,subscribers_count,last_year_commits,contributors,closed_issues,pull_requests
0,free-programming-books,EbookFoundation,359735,63576,Python,2013-10-11T06:50:37Z,2025-06-21T02:09:07Z,19483,359735,45,Organization,37,virtual,5000,339,432,129,15
1,public-apis,public-apis,351991,37004,Python,2016-03-20T23:49:42Z,2025-06-21T02:06:40Z,4771,351991,492,Organization,1,NaN,4316,5,427,159,490
2,system-design-primer,donnemartin,306925,50727,Python,2017-02-26T16:15:28Z,2025-06-21T02:06:11Z,11239,306925,498,User,27,"Washington, D.C.",5000,4,113,18,256
3,awesome-python,vinta,247255,25843,Python,2014-06-27T21:00:06Z,2025-06-21T02:08:14Z,6769,247255,486,User,20,Taiwan,5000,1,368,28,471
4,Python,TheAlgorithms,201541,46909,Python,2016-07-16T09:44:01Z,2025-06-21T01:50:56Z,15391,201541,397,Organization,44,India,5000,181,454,334,329


## ❌ Tratamento de Dados Ausentes (Missing Data)

### Diagnóstico de Ausências
Identificação e quantificação sistemática de valores faltantes em todas as colunas.

**Estratégias de Tratamento:**
- **`language`:** Exclusão de registros (dados críticos para análise)
- **`owner_location`:** Imputação com "Not informed" (dado opcional)

### Justificativas Técnicas:
- **Linguagem é fundamental** para categorização e análise comparativa
- **Localização é complementar** e pode ser tratada como "não informado"

In [5]:
# Check for and sum the number of missing (null) values for each column.
df.isnull().sum()

name                     0
owner                    0
stars                    0
forks                    0
language                 1
created_at               0
updated_at               0
size_kb                  0
watchers_count           0
open_issues              0
owner_type               0
owner_public_repos       0
owner_location        3969
subscribers_count        0
last_year_commits        0
contributors             0
closed_issues            0
pull_requests            0
dtype: int64

In [7]:
# Calculate and print the proportion of null values for 'language' and 'owner_location' columns.
language_null_prop = (df['language'].isnull().sum() / len(df)) * 100
location_null_prop = (df['owner_location'].isnull().sum() / len(df)) * 100

print('Proporção de nulos:')
print(f'Atributo language: {language_null_prop:.2f}%')
print(f'Atributo owner_location: {location_null_prop:.2f}%')

Proporção de nulos:
Atributo language: 0.01%
Atributo owner_location: 42.00%


In [8]:
# Handle missing values:
df = df[df['language'].notnull()]

df['owner_location'].fillna('Not informed', inplace=True)

/tmp/ipykernel_2576/474970543.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['owner_location'].fillna('Not informed', inplace=True)


In [9]:
df.isnull().sum()

name                  0
owner                 0
stars                 0
forks                 0
language              0
created_at            0
updated_at            0
size_kb               0
watchers_count        0
open_issues           0
owner_type            0
owner_public_repos    0
owner_location        0
subscribers_count     0
last_year_commits     0
contributors          0
closed_issues         0
pull_requests         0
dtype: int64

## 🔄 Eliminação de Duplicatas

### Detecção de Redundâncias
Identificação de registros completamente idênticos que podem distorcer análises estatísticas.

**Processo:**
1. **Identificação:** Localização de linhas duplicadas
2. **Análise:** Distribuição por linguagem para entender padrões
3. **Remoção:** Eliminação mantendo apenas registros únicos

**Impacto:** Redução do viés estatístico e melhoria na qualidade das análises

In [10]:
# Identify and count completely duplicate rows in the DataFrame.
df_duplicated = df[df.duplicated()]

print(f"Quantidade de duplicatas completas: {df_duplicated.shape[0]}")

Quantidade de duplicatas completas: 348


In [11]:
df_duplicated['language'].value_counts()

language
C       347
Java      1
Name: count, dtype: int64

In [12]:
# Remove all identified duplicate rows from the DataFrame.
df = df.drop_duplicates()

print(f'Após exclusão de dados duplicados: {len(df)} registros')

Após exclusão de dados duplicados: 9101 registros


## 🔍 Validação de Consistência de Dados

### Verificação de Valores Negativos
Auditoria de colunas numéricas para identificar valores fisicamente impossíveis ou inconsistentes.

**Colunas Verificadas:**
- Métricas de popularidade (stars, forks, subscribers)
- Indicadores de atividade (commits, contributors, issues)
- Dados temporais e de tamanho

### Validação de Formatos de Data
Conversão e validação de colunas temporais com tratamento de erros.

**Processo:**
- Conversão para `datetime` com `errors='coerce'`
- Identificação de valores não convertíveis
- Relatório de qualidade da conversão

### Auditoria de Dados Numéricos
Verificação de valores não numéricos em colunas esperadamente numéricas.

In [13]:
# Identify and list all numeric columns.
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Iterate through numeric columns to check for and display negative values.
for column in numeric_columns:
    negativos = df[df[column] < 0]
    qtd = negativos.shape[0]
    if qtd > 0:
        print(f"\nColuna '{column}' tem {qtd} valores negativos:")
        print(negativos[[column]].head(10))
    else:
        print(f"Coluna '{column}' não possui valores negativos.")

Coluna 'stars' não possui valores negativos.
Coluna 'forks' não possui valores negativos.
Coluna 'size_kb' não possui valores negativos.
Coluna 'watchers_count' não possui valores negativos.
Coluna 'open_issues' não possui valores negativos.
Coluna 'owner_public_repos' não possui valores negativos.
Coluna 'subscribers_count' não possui valores negativos.
Coluna 'last_year_commits' não possui valores negativos.
Coluna 'contributors' não possui valores negativos.
Coluna 'closed_issues' não possui valores negativos.
Coluna 'pull_requests' não possui valores negativos.


In [14]:
# Ensuring date columns are in datetime format
for col in ['created_at', 'updated_at']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

    n_nulos = df[col].isna().sum()
    print(f"Coluna '{col}' após conversão para datetime tem {n_nulos} valores que não foram convertidos")

Coluna 'created_at' após conversão para datetime tem 0 valores que não foram convertidos
Coluna 'updated_at' após conversão para datetime tem 0 valores que não foram convertidos


In [ ]:
# Checking for non-numeric entries in numeric columns
for col in numeric_columns:
    coerced = pd.to_numeric(df[col], errors='coerce')
    n_invalidos = coerced.isna().sum()
    print(f"Coluna '{col}' tem {n_invalidos} valores não numéricos")

Coluna 'stars' tem 0 valores não numéricos (NaN após conversão)
Coluna 'forks' tem 0 valores não numéricos (NaN após conversão)
Coluna 'size_kb' tem 0 valores não numéricos (NaN após conversão)
Coluna 'watchers_count' tem 0 valores não numéricos (NaN após conversão)
Coluna 'open_issues' tem 0 valores não numéricos (NaN após conversão)
Coluna 'owner_public_repos' tem 0 valores não numéricos (NaN após conversão)
Coluna 'subscribers_count' tem 0 valores não numéricos (NaN após conversão)
Coluna 'last_year_commits' tem 0 valores não numéricos (NaN após conversão)
Coluna 'contributors' tem 0 valores não numéricos (NaN após conversão)
Coluna 'closed_issues' tem 0 valores não numéricos (NaN após conversão)
Coluna 'pull_requests' tem 0 valores não numéricos (NaN após conversão)


## 📊 Análise de Dados Categóricos

### Padronização e Normalização
Investigação de variações e inconsistências em dados categóricos.

**Técnicas Aplicadas:**
- **Normalização:** Conversão para lowercase e remoção de espaços
- **Contagem de Frequências:** Identificação de variações e duplicatas semânticas
- **Detecção de Padrões:** Análise de consistência em categorias

### Tratamento Especial de Localizações
Exportação detalhada de dados de localização para análise geográfica posterior.

In [16]:
# Identify categorical columns.
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# For each categorical column, print unique values and their normalized counts.
for col in categorical_columns:
    print(f"\nValores únicos e suas contagens na coluna '{col}':")
    valores_unicos = df[col].dropna().unique()

    # Normalizing to lower case and removing spaces to identify variations.
    valores_normalizados = [str(v).strip().lower() for v in valores_unicos]
    contagem = pd.Series(valores_normalizados).value_counts()

    print(contagem.head(20))


Valores únicos e suas contagens na coluna 'name':
leetcode      3
tv            2
pulse         2
cameraview    2
icecream      2
gifski        2
menu          2
eureka        2
cardslider    2
java          2
schedule      2
ignite        2
time          2
surge         2
glance        2
nuklear       2
iris          2
shadow        2
skip          2
ios           2
Name: count, dtype: int64

Valores únicos e suas contagens na coluna 'owner':
stasel                  1
ebookfoundation         1
public-apis             1
donnemartin             1
vinta                   1
thealgorithms           1
significant-gravitas    1
automatic1111           1
cocoapods               1
hearthsim               1
xjbeta                  1
slazyk                  1
marioiannotta           1
radex                   1
kaandedeoglu            1
alexeybelezeko          1
yeahdongcn              1
venmo                   1
jiritrecak              1
pixel16                 1
Name: count, dtype: int64

Valo

In [ ]:
unique_locations = df['owner_location'].dropna().unique()

unique_location_normalized = [str(v).strip().lower() for v in unique_locations]
contagem_df = pd.Series(unique_location_normalized).value_counts().reset_index()
contagem_df.columns = ['location', 'count']

contagem_df.to_csv('../dados/location_counts.csv', index=False)

## 🗑️ Remoção de Colunas Redundantes

### Otimização do Dataset
Eliminação de colunas com informações duplicadas ou baixo valor analítico.

**Colunas Removidas:**
- **`owner_location`:** Alta cardinalidade e muitos valores faltantes
- **`watchers_count`:** Redundante com outras métricas de engajamento

**Justificativa:** Simplificação do dataset mantendo informações essenciais

In [ ]:
df = df.drop(['owner_location'], axis=1)
df = df.drop(['watchers_count'], axis=1)
df

,name,owner,stars,forks,language,created_at,updated_at,size_kb,open_issues,owner_type,owner_public_repos,subscribers_count,last_year_commits,contributors,closed_issues,pull_requests
0,free-programming-books,EbookFoundation,359735,63576,Python,2013-10-11 06:50:37+00:00,2025-06-21 02:09:07+00:00,19483,45,Organization,37,5000,339,432,129,15
1,public-apis,public-apis,351991,37004,Python,2016-03-20 23:49:42+00:00,2025-06-21 02:06:40+00:00,4771,492,Organization,1,4316,5,427,159,490
2,system-design-primer,donnemartin,306925,50727,Python,2017-02-26 16:15:28+00:00,2025-06-21 02:06:11+00:00,11239,498,User,27,5000,4,113,18,256
3,awesome-python,vinta,247255,25843,Python,2014-06-27 21:00:06+00:00,2025-06-21 02:08:14+00:00,6769,486,User,20,5000,1,368,28,471
4,Python,TheAlgorithms,201541,46909,Python,2016-07-16 09:44:01+00:00,2025-06-21 01:50:56+00:00,15391,397,Organization,44,5000,181,454,334,329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9445,YNSearch,younatics,1194,101,Swift,2017-04-16 05:59:12+00:00,2025-06-19 15:25:24+00:00,6384,3,User,60,24,0,5,0,0
9446,ReactKit,ReactKit,1194,40,Swift,2014-09-29 14:09:19+00:00,2025-04-20 09:17:41+00:00,1131,11,Organization,5,46,0,3,0,0
9447,CardSlider,saoudrizwan,1191,93,Swift,2017-02-27 21:05:55+00:00,2025-06-19 15:24:59+00:00,109,3,User,27,26,0,2,0,2
9448,VisualProgrammingLanguage,NathanFlurry,1190,45,Swift,2018-04-01 13:29:12+00:00,2025-06-08 00:26:17+00:00,35950,1,User,46,59,0,2,0,1


## 💾 Persistência do Dataset Limpo

Exportação do dataset processado para análises subsequentes.

**Arquivo Gerado:** `github_repos_limpo.csv`
**Melhorias Obtidas:**
- ✅ Eliminação de dados faltantes críticos
- ✅ Remoção de duplicatas
- ✅ Validação de consistência
- ✅ Otimização estrutural

In [25]:
df.to_csv('../dados/github_repos_limpo.csv', index=False)